In [8]:
import pandas as pd
import csv
import numpy as np
#from openai import OpenAI
import os
from bs4 import BeautifulSoup
import requests
from datetime import datetime
current_time = datetime.now().time()
from PIL import Image
import pytesseract
import requests
from io import BytesIO
#pytesseract.pytesseract.tesseract_cmd = r'"C:\Program Files\Tesseract-OCR\tesseract.exe"'


In [9]:
from textblob import TextBlob
from textblob.exceptions import TranslatorError




def translate(text_to_translate):
    
    blob = TextBlob(text_to_translate)
    
    # #Translate the text to another language 
    # translated_text = translator.translate(text_to_translate, dest='el')
    try:
        return blob.translate(from_lang='en', to='el')
    except TranslatorError as e:
    # Handle NotTranslated exception
        print(text_to_translate)
        print(f"Translation not possible: {e}")







In [83]:
# #preprocessing steps 

# df4  = pd.read_csv('test1.csv')

# df4['text_embedding'] = df4['text_embedding'].apply(lambda x: x.replace("[","").replace("]",""))

# df4['text_embedding'] = df4['text_embedding'].apply(lambda x: np.array(x.split(',')).astype('float'))


df = pd.DataFrame(columns=['id','claim','article_title','author','publish_date','harvest_date','url','label', 'article_text'])


In [6]:
#Harvest method for greekhoaxes.gr
def harvest(urls, df):
    for url in urls:
        print(url)
        html_text = requests.get(url).text
        soup = BeautifulSoup(html_text, 'html.parser')
        
        conclusion_string = ''
        article_text=''

        #harvest the date
        date = soup.find('div', class_ = 'blog-date').text

        #harvest the claim
        try:
            claim = soup.find('div', class_ = 'Claim')
            claim_text = claim.find_next('p').text

        except AttributeError as e:

            print(f"Attribute Error: {e}")
            continue
    
        
        #harvest the title
        title = soup.find('div', class_ = 'content-bottom')
        title_text = title.find_next('h1').get_text()

        #harvest the author 
        author = soup.find('a', class_ = 'author')
        author_text = author.get_text()

        # harvest the conclusion
        conclusion = soup.find('div', class_='conclusion')
        con_text = conclusion.find_next('p').get_text()

#         print(f'''Date: {date} 
# Claim: {claim_text}
# Title: {title_text}
# Author: {author_text}
# Conclusion: {con_text}
# URL: {url}''')

        #harvest the article text
        article = soup.find('h3',class_='wp-block-heading')
        if(article is None):
            article = soup.find('h4',class_='wp-block-heading')
            if(article is None):
                article = soup.find('h5',class_='wp-block-heading')
                if(article is None):
                    article = soup.find('h2',class_='wp-block-heading')
        try:
            article_text = article.find_next_siblings('p')
        except AttributeError as e:
            print(f"AttributeError: {e}")
        #print('Content:')
        for text in article_text:
            content = text.get_text().strip()
            if content:
                # print(content)
                conclusion_string += content


        #add the harvested data to the dataframe

        data = {'id': len(df), 'claim': claim_text, 'article_title': title_text, 'author': author_text, 'publish_date': date, 
                'harvest_date':current_time , 'url': url, 'article_text': conclusion_string, 'label': 0 }
        
        df.loc[len(df)] = data



        print()
      
        

In [7]:
#Harvesting greekhoaxes.gr
page_numbers = 10
pages = list()

html_text = requests.get('https://www.ellinikahoaxes.gr/category/kathgories/fake-news/').text
soup = BeautifulSoup(html_text, 'html.parser')
urls = soup.find_all('a', class_="btn btn-secondary")


#find the first page
current_page = soup.find('span', class_='page-numbers current')
print(f"Collecting urls from page1: https://www.ellinikahoaxes.gr/category/kathgories/fake-news/")
url_list = [url.get('href') for url in urls]
harvest(url_list, df)

#find the next page to harvest
current_page = soup.find('a', class_='page-numbers')


#find next pages to harvest depending on the page number variable
#after the page #2, we follow the same procedure recurcively
for i in range(page_numbers):

    
    print(f"Collecting urls from page: {current_page.get('href')}")
   
    
    #find next page
    html_text2 = requests.get(current_page.get('href')).text
    soup = BeautifulSoup(html_text2, 'html.parser')
    
    #recursively find its the next page each time
    current_page = soup.find('span', class_='page-numbers current')
    next_page = current_page.find_next('a', class_ = 'page-numbers')
    current_page = next_page
  
    urls = soup.find_all('a', class_="btn btn-secondary")
  
    url_list = [url.get('href') for url in urls]
    harvest(url_list, df)

    # for url in url_list:
    #     print(url)




https://www.ellinikahoaxes.gr/2023/12/12/psevdes-oti-o-oukranos-proedros-zelenski-agorase-dyo-giot-axias-75ek-dolarion/

https://www.ellinikahoaxes.gr/2023/12/09/paraskiniaka-plana-tainias-parousiazontai-psevdos-os-prospatheia-propagandas-ton-palaistinion/

https://www.ellinikahoaxes.gr/2023/11/17/gaza-parliament-demolished-video-false/

https://www.ellinikahoaxes.gr/2023/11/14/pakistani-antikythera-500-euro-fake-news/

https://www.ellinikahoaxes.gr/2023/11/13/vinteo-den-apeikonizei-dimosiografo-pou-synelifthi-sto-israil/

https://www.ellinikahoaxes.gr/2023/11/10/parapliroforisi-schetika-me-tin-ypotithemeni-epithesi-stin-aftokinitopobi-tou-igeti-tis-palaistinis-machmount-abas/

https://www.ellinikahoaxes.gr/2023/11/01/ochi-to-evropaiko-dikastirio-anthropinon-dikaiomaton-den-akyrose-tin-symfonia-ton-prespon/

https://www.ellinikahoaxes.gr/2023/11/01/ochi-to-inomeno-vasileio-den-psifise-nomous-gia-klimatika-lockdowns/

https://www.ellinikahoaxes.gr/2023/10/27/pseftiki-synentefxi-me-robot

In [8]:
harvest(['https://www.ellinikahoaxes.gr/2023/07/29/kyriakos-velopoulos-i-dana-levy-pou-egine-gideon-levy-i-epimenei-sto-fake-news-gia-ta-ilyuishin-me-neo-fake-news-wip/'],df)

https://www.ellinikahoaxes.gr/2023/07/29/kyriakos-velopoulos-i-dana-levy-pou-egine-gideon-levy-i-epimenei-sto-fake-news-gia-ta-ilyuishin-me-neo-fake-news-wip/



In [9]:
def harvest_2(urls, df, gr):
    for url in urls:

        html_text = requests.get(url).text
        soup = BeautifulSoup(html_text, 'html.parser')
        article_text = ''
        author_text = ''
        title = soup.find('h1', class_ = 'content-title').text.strip()
        date = soup.find('span', class_ = 'date').text
        authors = soup.find('span', class_ = 'meta-author')
        for author in authors:
            if(author_text!=''):
                author_text = author_text + author.text 
            else:
                author_text = author.text 

            

        article = soup.find_all('div', class_ = 'article-entry clearfix')
        for p in article:
            article_text += p.text.replace("'","")
        
        if(gr==False):
            try:
                article_text = translate(article_text).strip()
            except TypeError as e:
                print(f"Type: {e}")
                article_text = ''

#         print(f'''
# Title: {title}
# Date: {date}
# Authors: {author_text}
# Claim: {title}
# Main_article: {article_text}
# ''')
    print(article_text)
        
    if(gr):
        data = {'id': len(df), 'claim': title, 'article_title': title, 'author': author_text, 'publish_date': date, 
                    'harvest_date':current_time , 'url': url, 'article_text': str(article_text).strip(), 'label': 1 }
    else:
        data = {'id': len(df), 'claim': translate(title), 'article_title':  translate(title), 'author':  translate(author_text), 'publish_date': date, 
                    'harvest_date':current_time , 'url': url, 'article_text': article_text, 'label': 1 }
        
    df.loc[len(df)] = data

In [10]:

page_numbers_2 = 10



URL = 'https://factcheckgreek.afp.com'

html_text = requests.get('https://factcheckgreek.afp.com/list').text
soup = BeautifulSoup(html_text, 'html.parser')

#we set the next_page_url to the main page at first
next_page_url = 'https://factcheckgreek.afp.com/list' 

#we iterate through all the pages of the site
for i in range(page_numbers_2):


    print(f'Getting urls from page: {i+1}')
    print(f'harvesting url: {URL+next_page_url}')

    
    #get the urls of the main page
    urls = soup.find_all('div' , class_ = 'card')

    #harvest the data for each url 
    url_list = list()
    for url in urls:
        
        url_text = url.find('a').get('href')
        url_text = URL + url_text
        url_list.append(url_text)
        #print(url_text)

    #harvest function for main_url
    harvest_2(url_list,df, True)

    #move to the next page
    current_page = soup.find('li', class_='page-item active')
   
    next_page = current_page.find_next_sibling('li', class_ = 'page-item')
    
    next_page_url = next_page.find('a').get('href')
    

    html_text = requests.get(URL+next_page_url).text
    soup = BeautifulSoup(html_text, 'html.parser')
    












Getting urls from page: 1
harvesting url: https://factcheckgreek.afp.comhttps://factcheckgreek.afp.com/list
Ο πυκοντικός Ι -Ι. Ο -εζόμενος Δυΐνννςς Τ, ο γιοντ, ο ίδιος του Ιο. Έ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ς ΟΜΙΜΙΣΟΣ ΑΛΟΚΟΛΟΣ »ΣΤΟ ΙΟΓΑΛΛΛΛΛΛΛΛΛΛΛΟΛΗΕΜΕΡ. Ομοί, ο Χρόννιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιιι, ΠΛΗΝΑΝΥΝΗ ΤΟΥΣ ΑΠΟΝΣΣΜΑ, ΤΩΝΜΟΤΩΝΤΟΥΝΤΟΥΝΤΟΥΝΤΟΥΝΤΟΥΝΤΟΥΝΤΟΥΝΤΟΥΝΩΝΩΝ. ΟΙ ΑΝΤΑΔΙΙΙΙΙΙΙΙΙΙΙΙΙΙΙΙΙΝΙΩΝΙΩΝΙΩΝΙΩΝΩΝΩΝΩΝΩΝΩΝΩΝΩΝΩΝΩΝΩΝΩΝΩΝΩΝΩΩΝΩΝΩΩΝΩΝΩΩΝΩΝΩΩΝΩΝικός. Η εδάφη χρωσία του Ι -η Ι -Ι -Ιεν -Ι -Ι -Ι -Ι. ΟΙΝΑΛΛΛΑΝΙΣΙΣΙΣΙΩΝ, ΑΛΑΚΝΑΝΗΝΩΣΙΑΣΕΩΣΩΣΗ ΕΝΥΚΩΝΩΣΤΩΝΩΣΤΩΝΩΣΤΩΝΩΣΗΚΩΝΩΣΤΩΝΩΔΙΩΝΩΔΙΩΝΩΔΩΚΩΝΩΣΤΩΝΩΣΩ Ο ΠΟΙΤΙΝΝΙΝΙ, ο ΠΟΝΟΣΙΣΟΣΙΩΝΙΩΝΙΩΝΤΟΣ. Ο Ααϊν -ΝΟΝΤΥΝΟΥΝΤΥΝΙΔΙΔΙΔΙΔΙΩΝΑ, ΑΙΝΑΔΙΣΣΟΚΤΗΣ 7 ΕΝΥΛΙ, ΜΟΝΕΩΝΗ, ΕΝΥΩΝΗΕΩΣΕΩΝΩΣΤΩΝΗΕΩΣΕΩΝΗΕΩΣΕΩΝΗΥΩΝ Κο -βομυπαϊμ, ο Λωϊδ -Ι, ο Κουϊρ. ΕΙΜΑΝΤΙΝΗΝΗ ΜΑΧΑΔΩΣΩΣΗ, ΚΥΩΣΑ, ΕΝΤΟΔΟΙΝΔΟΤΟΣ ΕΝΥΠΟΤΟΣ ΕΥΤΟΣΤΟΣΥΣΟΟΣΣΣΟΣΥΣΩΣΩΣΩΣΩΣΩΣΩΣΩΣΩΣΩΣΩΣΩΣΩΣΩΣΩΣΩΣΩΣας 20 o -ROS. Στις 2 Νοεμβρίου, το υπουργε

In [11]:
#Harvest afp urls for other regions other than greece

regions = ['AFRICA','ASIA-PACIFIC','NORTH AMERICA','EUROPE','LATIN AMERICA','MIDDLE EAST']
# regions=['AFRICA']
site_url = 'https://factcheck.afp.com/'
html_text = requests.get(site_url, timeout=2000).text
soup = BeautifulSoup(html_text, 'html.parser')
region_urls = soup.find_all('a', class_='btn btn-outline-dark')
region_urls_text = list()
for region in region_urls:
    if(region.text in regions):
        #print(site_url+region.get('href'))
        region_urls_text.append(site_url+region.get('href'))


#use the same method as above for each region url

for region_url in region_urls_text:

    html_text = requests.get(region_url).text
    soup = BeautifulSoup(html_text, 'html.parser')

    URL = site_url[:-1]

    html_text = requests.get(region_url, timeout=2000).text
    soup = BeautifulSoup(html_text, 'html.parser')

    #we set the next_page_url to the main page at first
    next_page_url = region_url

    #we iterate through all the pages of the site
    for i in range(20):


        print(f'Getting urls from page: {i+1}')
        print(f'harvesting url: {next_page_url}')

        
        #get the urls of the main page
        urls = soup.find_all('div' , class_ = 'card')

        #harvest the data for each url 
        print('SUB-Urls:')
        url_list = list()
        for url in urls:
            
            url_text = url.find('a').get('href')
            url_text = URL + url_text
            url_list.append(url_text)
            print(url_text)

        #harvest function for main_url
        harvest_2(url_list,df, False)

        #move to the next page
        current_page = soup.find('li', class_='page-item active')
    
        next_page = current_page.find_next_sibling('li', class_ = 'page-item')
        
        next_page_url = next_page.find('a').get('href')
        

        html_text = requests.get(URL+next_page_url).text


    



Getting urls from page: 1
harvesting url: https://factcheck.afp.com//list/all/all/all/38558/19
SUB-Urls:
https://factcheck.afp.com/doc.afp.com.347B98J
https://factcheck.afp.com/doc.afp.com.34794P9
https://factcheck.afp.com/doc.afp.com.347B7Y2
https://factcheck.afp.com/doc.afp.com.347799J
https://factcheck.afp.com/doc.afp.com.34768DR
https://factcheck.afp.com/doc.afp.com.346F4CK
https://factcheck.afp.com/doc.afp.com.346F68V
https://factcheck.afp.com/doc.afp.com.346F762
https://factcheck.afp.com/doc.afp.com.34689T4
https://factcheck.afp.com/doc.afp.com.344N7AK
https://factcheck.afp.com/doc.afp.com.344C9JA
https://factcheck.afp.com/doc.afp.com.343F9CP
https://factcheck.afp.com/doc.afp.com.344E2WJ
https://factcheck.afp.com/doc.afp.com.344D8G9
https://factcheck.afp.com/doc.afp.com.34488BK
Οι θεωρίες συνωμοσίας σχετικά με την αυξανόμενη τάση των εμφυτευμάτων τσιπ αφθονούν, εκμεταλλευόμενοι τους φόβους που περιβάλλουν την αναδυόμενη τεχνολογία. Πρόσφατα, ένα κλιπ εμφανίστηκε στο διαδίκτυο ισχ

In [12]:
harvest_2(['https://factcheck.afp.com/doc.afp.com.344G7CK'],df,False)


Κατασκευασμένες διαφημίσεις που περιορίζουν τα "Get Rich Quick" επενδυτικά προγράμματα που διαθέτουν έναν γνωστό δημοσιογράφο της Νότιας Κορέας, Sohn Suk-Hee, έχουν κυκλοφορήσει επανειλημμένα στα κοινωνικά μέσα ενημέρωσης στη χώρα. Ένα βίντεο, που βλέπει χιλιάδες φορές, έχει χειριστεί χρησιμοποιώντας τεχνητή νοημοσύνη, στους εμπειρογνώμονες στο AFP. Μια ξεχωριστή "ειδησεογραφική έκθεση" έχει τεθεί σε εξέλιξη από την ιστοσελίδα του Chosun Ilbo, μιας μεγάλης εφημερίδας της Νότιας Κορέας. Από τις 30 Νοεμβρίου 2023, ο Sohn δεν έχει προωθήσει κανένα επενδυτικό σύστημα. "Οι τράπεζες φοβούνται αυτή τη νέα μέθοδο και προσπαθούν σταδιακά να την απαγορεύσουν", διαβάζει τα βίντεο της κορεατικής λεζάντας στο Facebook. Κέρδισε περισσότερες από 12.000 προβολές αφού μοιράστηκε στις 20 Νοεμβρίου 2023. Το κλιπ των 30 δευτερολέπτων φαίνεται να δείχνει συνταξιούχο δημοσιογράφο της Νότιας Κορέας Sohn Suk-Hee προωθώντας "μια καινοτόμο νέα επενδυτική πλατφόρμα με έδρα την ΑΙ που άνοιξε την πορεία προς την ο

In [13]:
#harvest for fullfacts.org
def harvest_3(urls,df):
    for url in urls:
        print(url)
        article_text=''
        html_text = requests.get(url, timeout=2000).text
        soup = BeautifulSoup(html_text, 'html.parser')
        title = soup.find('h1', class_='mb-3 highlight-js').text
        try:
            #harvest the variables needed
            claim = soup.find('p', class_='card-text').text
            date = soup.find('div', class_='published-at mb-2').text
            author = soup.find('cite').text
            article = soup.find_all('div', class_='cms-content')
            
            
            for text in article:
                article_text+=text.text.strip()

            #translate article to greek
            article_text = translate(article_text).strip().replace("'","")

            #harvest label
            label = soup.select_one('p.m-0 > a:nth-of-type(2)').text
            print(label)
            if not label :
                label = 'True'

            data = {'id': len(df), 'claim': translate(claim), 'article_title':  translate(title), 'author':  author, 'publish_date': date, 
                    'harvest_date':current_time , 'url': url, 'article_text': article_text, 'label': translate(label) }
            df.loc[len(df)] = data


        
        except TypeError as e:
            print(f"Type Error: {e}")
            article_text = ''

        except AttributeError as e:
            print(f"AttributeError: {e}")
            continue
       
        
        #label = soup.find_all('p', class_='m-0')
        
            

        #     print(f'''
        #  Title: {title}
        #  Claim: {claim}
        #  Date: {date}
        #  Author: {author}
        #  Label: {label}
        #  Text: {article_text}''')







       
        
        

        


        
        


In [14]:
    
harvest_3(['https://fullfact.org/immigration/rishi-sunak-100000-migrants-repeat/'],df)

https://fullfact.org/immigration/rishi-sunak-100000-migrants-repeat/
AttributeError: 'NoneType' object has no attribute 'text'


In [15]:
#Fullfact.org harvest

static_url = 'https://fullfact.org/latest/?page='
main_url = 'https://fullfact.org'
idx=0
pages = 10 #20/63
batches=6

for j in range(batches):
    
    for i in range(idx,pages):
        print('Page num:', i)
        url_list = list()
        url = static_url + str(i+1)

        print(f"Harvesting url: {url}")
        
        html_text = requests.get(url, timeout=2000).text
        soup = BeautifulSoup(html_text,'html.parser')
        sub_urls = soup.find_all('a', class_ = 'btn btn-brand-neutral-black stretched-link')
        print('Sub-urls:')
        for url in sub_urls:
            # print(main_url+url.get('href'))
            url_list.append(main_url+url.get('href'))
            # harvest_3(main_url+url.get('href'),df)
        harvest_3(url_list,df)

    pages+=10
    idx+=10


Page num: 0
Harvesting url: https://fullfact.org/latest/?page=1
Sub-urls:
https://fullfact.org/online/ulez-lez-congestion-charge-christmas-day/
partly false
https://fullfact.org/news/gov-website-mayoral-elections/
AttributeError: 'NoneType' object has no attribute 'text'
https://fullfact.org/health/acesfulfame-k-methylene-chloride/
missing context
https://fullfact.org/online/missing-man-hoax-appeals/
false
https://fullfact.org/online/missing-dog-and-puppy-hoax-post/
false
https://fullfact.org/online/lee-rigby-killer-michael-adebolajo-not-killed/
false
https://fullfact.org/online/bus-fire-Bradford-false-claim-London-clean-air-zone/
false
https://fullfact.org/news/hansard-mary-glindon-update/
AttributeError: 'NoneType' object has no attribute 'text'
https://fullfact.org/online/cargo-ship-attack-footage/
partly false
https://fullfact.org/news/badenoch-manufacturing-britain-france/
AttributeError: 'NoneType' object has no attribute 'text'
https://fullfact.org/health/boris-johnson-covid-inq

In [22]:
df.to_csv('web_scraping', index=False)


In [23]:
df.shape


(883, 9)

In [12]:

df2 = pd.DataFrame(columns=['id','claim','article_title','author','publish_date','harvest_date','url','label', 'article_text'])


In [13]:
#harvest pages for snopes
main_url = 'https://www.snopes.com/fact-check/?pagenum='
pages = 50
url_list = list()

for i in range(1,pages+1):
    html_text = requests.get(f'https://www.snopes.com/fact-check/?pagenum={i}', timeout=100).text
    soup = BeautifulSoup(html_text,'html.parser')
    urls = soup.find_all('a', class_="outer_article_link_wrapper")
    
    for url in urls:
        url_list.append(url.get('href'))
    

In [14]:
#harvest data for snopes.com
for url in url_list:
    print(f"Harvesting url: {url}")
    html_text = requests.get(url, timeout=100).text
    soup = BeautifulSoup(html_text,'html.parser')
    article_text = ''
    #get title
    title = soup.find('section', class_='title-container')
    title_text = title.find('h1').text
   
    
    try:
        #get claim
        claim = soup.find('div', class_='claim_cont').text.strip()
        #get label
        label = soup.find('div', class_='rating_title_wrap').text.strip()
        label_text = label.replace("About this rating","").strip()
    except AttributeError as e:
        print(f"AttributeError: {e}")
        continue
    #get author
    author = soup.find('h3', class_='author_name').text.strip()
    #get date
    date = soup.find('h3', class_='publish_date').text.replace('Published ','')
    #get article
    article = soup.find_all('article',id='article-content' )
    for ar in article:
        article_text+=ar.text.replace("\t","").strip()

    #translation to greek
    title_text = translate(title_text)
    article_text = translate(article_text)
    claim = translate(claim)

    #if the translation was unsuccessful
    if( (not title_text) | (not article_text) | (not claim) ):
        continue
    else:

        data = {'id': len(df2), 'claim': claim, 'article_title':  title_text, 'author':  author, 'publish_date': date, 
                'harvest_date':current_time , 'url': url, 'article_text': article_text, 'label': label_text}
        df2.loc[len(df2)] = data
    
        
    
    

Harvesting url: https://www.snopes.com/fact-check/young-girls-epsteins-island/
Harvesting url: https://www.snopes.com/fact-check/lost-child-in-denmark/
Harvesting url: https://www.snopes.com/fact-check/pics-flower-ice-formations/
Harvesting url: https://www.snopes.com/fact-check/trump-speech-jan6/
Harvesting url: https://www.snopes.com/fact-check/trump-perry-school-shooting/
Harvesting url: https://www.snopes.com/fact-check/natalie-portman-female-directors/
Harvesting url: https://www.snopes.com/fact-check/bill-clinton-make-mistakes/
Harvesting url: https://www.snopes.com/fact-check/obaid-chinoy-star-wars/
Harvesting url: https://www.snopes.com/fact-check/bush-painting-epstein/
Harvesting url: https://www.snopes.com/fact-check/taylor-swift-jeffrey-epstein-pic/
Harvesting url: https://www.snopes.com/fact-check/putin-mcgregor-warning/
Harvesting url: https://www.snopes.com/fact-check/george-washington-dinosaurs-discovered/
Harvesting url: https://www.snopes.com/fact-check/kim-putin-toast

In [16]:

df3 = df2.drop_duplicates(subset=['claim'])
df3 = df3.reset_index(drop=True)
df2.shape,df3.shape

df2.to_csv('data/snopes_df.csv')

In [204]:
df3 = pd.read_csv('snopes_df.csv')

In [200]:
df3 = df3.iloc[:,1:]

In [68]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException


URL = 'https://www.reuters.com/fact-check/'
url_list = list()

driver = webdriver.Chrome()
driver.get(URL)

reject_xpath = '//*[@id="onetrust-reject-all-handler"]'
button_xpath = reject_xpath  # Adjust the XPath to the button
button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, button_xpath)))
button.click()



for i in range(19):
    # Locate and click the button that triggers dynamic content
    button_xpath = '//*[@id="main-content"]/div[1]/div/div[1]/div/div/button'  # Adjust the XPath to the button
    button = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, button_xpath)))
    button.click()
           
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')


list_elements = soup.find_all('li', class_='feed-list__item__Up-UF')

for element in list_elements:
    
    href = element.find('div')
    href_text = href['href']
    print('https://www.reuters.com' + href_text)
    url_list.append('https://www.reuters.com' + href_text)

driver.quit()


https://www.reuters.com/fact-check/bill-gates-didnt-order-governments-use-only-gm-food-summit-2023-12-19/
https://www.reuters.com/fact-check/al-jazeera-not-banned-uk-television-2023-12-19/
https://www.reuters.com/fact-check/sentence-killing-children-is-morally-wrong-not-listed-hate-symbol-by-adl-2023-12-19/
https://www.reuters.com/fact-check/image-does-not-show-authentic-homework-assignment-about-bill-clinton-2023-12-19/
https://www.reuters.com/fact-check/iss-footage-is-not-filmed-nasa-training-pool-2023-12-15/
https://www.reuters.com/fact-check/amazon-hub-addresses-were-added-accounts-error-says-company-2023-12-15/
https://www.reuters.com/fact-check/obama-collage-includes-photos-that-are-taken-out-context-altered-2023-12-15/
https://www.reuters.com/fact-check/video-truck-driving-into-protest-predates-la-demonstration-2023-2023-12-15/
https://www.reuters.com/fact-check/nasas-rs-25-is-rocket-launch-system-not-geoengineering-machine-2023-12-15/
https://www.reuters.com/fact-check/video-do

In [85]:






for url in url_list:
    print(url)
    driver = webdriver.Chrome()

    driver.get(url)

    reject_xpath = '//*[@id="onetrust-reject-all-handler"]'
    button_xpath = reject_xpath  # Adjust the XPath to the button
    button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, button_xpath)))
    button.click()

    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    title = soup.find('div', class_='default-article-header__heading__3cyKI').find('h1').text
    title = title.replace('Fact Check: ',"")

    date = soup.find('div', class_='default-article-header__heading__3cyKI').find('span', class_='date-line__date___kNbY').text

    article = soup.find_all('div', class_='article-body__content__17Yit')
    article_text=''

    for par in article:
        article_text +=par.text

    article_text = article_text.replace("Read more about our fact-checking work.Our Standards: The Thomson Reuters Trust Principles.","")


    try:
        
        title = translate(title)
        article_text = translate(article_text)
    except TranslatorError as e:
        print(f"Translation Error: {e}")

    data = {'id': len(df), 'claim': title, 'article_title':  title, 'author':  'Reuters Fact Check', 'publish_date': date, 
                'harvest_date':current_time , 'url': url, 'article_text': article_text, 'label': 1}
    df.loc[len(df)] = data

    driver.quit
   
  


https://www.reuters.com/fact-check/bill-gates-didnt-order-governments-use-only-gm-food-summit-2023-12-19/
https://www.reuters.com/fact-check/al-jazeera-not-banned-uk-television-2023-12-19/
https://www.reuters.com/fact-check/sentence-killing-children-is-morally-wrong-not-listed-hate-symbol-by-adl-2023-12-19/
https://www.reuters.com/fact-check/image-does-not-show-authentic-homework-assignment-about-bill-clinton-2023-12-19/
https://www.reuters.com/fact-check/iss-footage-is-not-filmed-nasa-training-pool-2023-12-15/
https://www.reuters.com/fact-check/amazon-hub-addresses-were-added-accounts-error-says-company-2023-12-15/
https://www.reuters.com/fact-check/obama-collage-includes-photos-that-are-taken-out-context-altered-2023-12-15/
https://www.reuters.com/fact-check/video-truck-driving-into-protest-predates-la-demonstration-2023-2023-12-15/
https://www.reuters.com/fact-check/nasas-rs-25-is-rocket-launch-system-not-geoengineering-machine-2023-12-15/
https://www.reuters.com/fact-check/video-do

In [90]:
df.to_csv('reuters.csv', index=False)

In [91]:
df_reuters = pd.read_csv('reuters.csv')

In [92]:
df_reuters.head()

,id,claim,article_title,author,publish_date,harvest_date,url,label,article_text
0,0,Ο Μπιλ Γκέιτς δεν διέταξε τις κυβερνήσεις να χ...,Ο Μπιλ Γκέιτς δεν διέταξε τις κυβερνήσεις να χ...,Reuters Fact Check,"December 19, 2023",00:03:35.441124,https://www.reuters.com/fact-check/bill-gates-...,1,Ένα άρθρο που ισχυριζόταν ότι ο Bill Gates διέ...
1,1,Ο Al Jazeera δεν απαγορεύτηκε από την τηλεόρασ...,Ο Al Jazeera δεν απαγορεύτηκε από την τηλεόρασ...,Reuters Fact Check,"December 19, 2023",00:03:35.441124,https://www.reuters.com/fact-check/al-jazeera-...,1,Οι ισχυρισμοί που διαδίδονται στα κοινωνικά μέ...
2,2,Η πρόταση «δολοφονίας των παιδιών είναι ηθικά ...,Η πρόταση «δολοφονίας των παιδιών είναι ηθικά ...,Reuters Fact Check,"December 19, 2023",00:03:35.441124,https://www.reuters.com/fact-check/sentence-ki...,1,Ένα στιγμιότυπο οθόνης που υποτίθεται ότι δείχ...
3,3,Η εικόνα δεν εμφανίζει αυθεντική εργασία για τ...,Η εικόνα δεν εμφανίζει αυθεντική εργασία για τ...,Reuters Fact Check,"December 19, 2023",00:03:35.441124,https://www.reuters.com/fact-check/image-does-...,1,Ένα έργο τέχνης που απεικονίζει μια εφευρεμένη...
4,4,Το υλικό ISS δεν γυρίζει στην πισίνα κατάρτιση...,Το υλικό ISS δεν γυρίζει στην πισίνα κατάρτιση...,Reuters Fact Check,"December 15, 2023",00:03:35.441124,https://www.reuters.com/fact-check/iss-footage...,1,Τα βίντεο και οι εικόνες συλληφθούν συνήθως απ...
